## Exercise 1

In [1]:
def sim_tanimoto(prefs,p1,p2):
    
    si={}
    for item in prefs[p1]: 
        if item in prefs[p2]:
            si[item]=1

    if len(si)==0:
        return 0
    
    n=len(si)
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
    den=sum1Sq + sum2Sq - pSum

    if den==0:
        return 0

    return pSum/den

## Exercise 2

In [2]:
from pydelicious import get_popular, get_userposts, get_urlposts

def initializeTagsDict(tags, count=5):
    tags_dict={}
    for tag in tags:
        tags_dict[tag]={}
    return tags_dict

def fillItems(user_dict):
    all_items={}
    # Find links posted by all users
    for user in user_dict:
        for i in range(3):
            try:
                posts=get_userposts(user)
                break
            except:
                print("Failed user "+user+", retrying")
                time.sleep(4)
        for post in posts:
            url=post['href']
            user_dict[user][url]=1.0
            all_items[url]=1
    # Fill in missing items with 0
    for ratings in user_dict.values( ):
        for item in all_items:
            if item not in ratings:
                ratings[item]=0.0

## Exercise 3

In [5]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5},
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0},
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [20]:
def sim_pearson(prefs,p1,p2):

    si={}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item]=1
    
    n=len(si)
    if n==0: return 0

    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
    num=pSum-(sum1*sum2/n)
    den=pow(((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n)), 0.5)

    if den==0: return 0
    return num/den

def topMatches(prefs,person,n=5,similarity=sim_pearson):
    
    scores=[(similarity(prefs,person,other),other) for other in prefs if other!=person]
    scores.sort()
    scores.reverse()
    return scores[0:n]

def pre_sim(prefs, n, similarity=sim_pearson):
    sim = {}
    for p1 in prefs:
        sim[p1] = topMatches(prefs, p1, n, similarity)
    
    return sim

def getRecommendations(prefs,person,similarity=sim_pearson):
  
    n = 5
    totals={}
    simSums={}
    sims = pre_sim(prefs, n)
    for pair in sims[person]:
        for item in prefs[pair[1]]:
            if item not in prefs[person] or prefs[person][item]==0:
                totals.setdefault(item,0)
                totals[item]+=prefs[pair[1]][item]*pair[0]
                simSums.setdefault(item,0)
                simSums[item]+=pair[0]
    
    rankings=[(total/simSums[item],item) for item,total in totals.items( )]
    rankings.sort()
    rankings.reverse()
    return rankings

getRecommendations(critics, 'Toby')

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]